In [2]:
from ultralytics import YOLO
import os
import gdown
import shutil
import yaml

### Вспомогательные функции

In [3]:
# Загрузить разметку с гугл диска
def load_pseudo_labels(
        dataset_name, #VOC
        ann_name, # Название разметки
        ann_postfix, # Постфикс разметки
        url # Урл, чтобы скачать (google disk)
        ):
    
    save_dir = f'../data/pseudo_labels/{dataset_name}'
    os.makedirs(save_dir, exist_ok=True)
    # Загрузка разметки
    filename = f'{save_dir}/{ann_name}.zip'
    gdown.download(url, filename, quiet = True, fuzzy = True)
    !unzip -d {save_dir} {filename} 

    # Копирование в папку с датасетом
    for sample in (['train', 'test', 'val']):
        lbs_source = f'{save_dir}/{ann_name}/{sample}'
        if os.path.exists(lbs_source) is False:
            print(f'{lbs_source} does not exisist')
            continue
        lbs_dest = f'../datasets/{dataset_name}/labels/{sample}_{ann_postfix}'
        shutil.copytree(lbs_source, lbs_dest)

# Подгрузить разметку в зависимости от эксперимента
def prepare_labels(dataset_name, # Название датасета
                   train_postfix=None, # Разметка, по умолчанию оригинальная [orig, GD]
                   val_postfix=None, # Разметка, по умолчанию оригинальная [orig, GD]
                   test_postfix=None # Разметка, по умолчанию оригинальная [orig, GD]
                   ):
    # Перезаписать разметку на разметку на выбранную
    for sample, postfix in zip(['train', 'val', 'test'], [train_postfix, val_postfix, test_postfix]):
        if postfix is None:
            postfix = 'orig'

        lbs_dest = f'../datasets/{dataset_name}/labels/{sample}'
        lbs_source = f'../datasets/{dataset_name}/labels/{sample}_{postfix}'
        !rm -rf {lbs_dest}
        shutil.copytree(lbs_source, lbs_dest)

# Чтение конфигурационного файла
def read_yaml_file(file_path):
    with open(file_path, 'r') as file:
        try:
            data = yaml.safe_load(file)
            return data
        except yaml.YAMLError as e:
            print("Error reading YAML file:", e)

### Загрузка разметки

In [3]:
# Загрузить разметку с гугл диска
# load_pseudo_labels(
#         dataset_name='VOC', #VOC
#         ann_name='annotations_nms_0.7_p_0.4', # Название разметки
#         ann_postfix='GD', # Постфикс разметки
#         url='https://drive.google.com/file/d/1K9-6IppbpJTnLivh80Or4Oj5tZUgd3ej/view?usp=sharing' # Урл, чтобы скачать (google disk)
#         )


### Запуск экспериментов

In [6]:
!ls runs/detect/orig_yolov8n_train

args.yaml			 PR_curve.png	   val_batch0_labels.jpg
confusion_matrix_normalized.png  R_curve.png	   val_batch0_pred.jpg
confusion_matrix.png		 results.csv	   val_batch1_labels.jpg
F1_curve.png			 results.png	   val_batch1_pred.jpg
labels_correlogram.jpg		 train_batch0.jpg  val_batch2_labels.jpg
labels.jpg			 train_batch1.jpg  val_batch2_pred.jpg
P_curve.png			 train_batch2.jpg  weights


In [3]:
%cd ..

/home/nikolay/Documents/Projects/Diploma/My repos/Pseudo_label_YOLO


/home/nikolay/Documents/Projects/Diploma/My repos/Pseudo_label_YOLO/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [20]:
import sys
sys.getsizeof(torch.tensor(1, dtype=torch.double))

80

In [22]:
!pip install torchsummary

  Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)


In [28]:
# YOLO('yolov8s.pt').parameters
sum(p.numel() for p in YOLO('yolov8s.pt').parameters())
# list(YOLO('yolov8s.pt').parameters())[0].dtype

11166560

In [10]:
#test
weight_path = 'notebooks/runs/detect/orig_yolov8n_train/weights/best.pt'
model =YOLO(weight_path)
validation_results = model.val(data='datasets/VOC/VOC_test.yaml', imgsz=640, batch=10)

Ultralytics YOLOv8.1.35 🚀 Python-3.10.6 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5938MiB)
Model summary (fused): 168 layers, 3009548 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/nikolay/Documents/Projects/Diploma/My repos/Pseudo_label_YOLO/datasets/VOC/labels/test.cache... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 496/496 [00:22<00:00, 22.43it/s]


                   all       4952      12032      0.731      0.425      0.542      0.367
             aeroplane       4952        285       0.72      0.515      0.596      0.416
               bicycle       4952        337      0.658      0.608       0.65      0.425
                  bird       4952        459      0.723      0.414      0.507      0.322
                  boat       4952        263       0.81     0.0164      0.327      0.185
                bottle       4952        469      0.345     0.0405      0.112      0.078
                   bus       4952        213          1     0.0119      0.121     0.0887
                   car       4952       1201      0.656      0.734      0.755      0.553
                   cat       4952        358      0.567      0.768      0.717      0.464
                 chair       4952        756      0.535      0.437      0.477      0.306
                   cow       4952        244      0.733      0.316      0.466      0.356
           diningtabl

In [6]:
model =YOLO('yolov8s.pt')
# validation_results = model.val(data='datasets/VOC/VOC_test.yaml', imgsz=640, batch=20)

In [14]:
validation_results.box

ultralytics.utils.metrics.Metric object with attributes:

all_ap: array([[ 0.00013981,  0.00013981,  0.00013981,  0.00012822,  0.00012249,  0.00010547,  9.4299e-05,  4.5777e-05,  2.5138e-05,   2.009e-05],
       [    0.92777,     0.91888,     0.90012,     0.87354,     0.84137,     0.80275,     0.70022,     0.55244,     0.38879,     0.17175],
       [  6.227e-05,   6.227e-05,   6.227e-05,  4.6732e-05,  3.1246e-05,  3.1246e-05,  3.1246e-05,  3.1246e-05,           0,           0],
       [ 0.00026401,  0.00026401,  0.00019866,  0.00019866,  0.00019866,  0.00012732,  0.00012732,  6.3423e-05,  6.3423e-05,  6.3423e-05],
       [  0.0002013,   0.0002013,   0.0002013,   0.0002013,   0.0002013,   0.0002013,   0.0002013,   0.0002013,   0.0002013,  0.00010044],
       [     0.9522,      0.9522,      0.9522,     0.94033,     0.93528,     0.93227,     0.90839,        0.86,     0.74901,     0.32313],
       [  0.0098487,   0.0098487,   0.0096827,   0.0091642,   0.0088394,   0.0083542,   0.0077109,  

In [12]:
validation_results.box.map50

0.0950899363558248

In [7]:
import torch

In [10]:
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_s

In [19]:
import torch

In [20]:
torch.save(validation_results, 'model_state_dict.pth')

TypeError: cannot pickle 'torch._C.Generator' object

In [36]:
# Save the file
pickle.dump(validation_results.box, file = open(f'{str(validation_results.save_dir)}/results.pickle', "wb"))
# Reload the file
# company1_reloaded = pickle.load(open("company1.pickle", "rb"))

In [37]:
reloaded = pickle.load(open(f'{str(validation_results.save_dir)}/results.pickle', "rb"))

In [41]:
reloaded.ap50

array([    0.59646,     0.64999,     0.50722,     0.32681,     0.11159,     0.12132,     0.75523,     0.71697,     0.47709,     0.46577,      0.5214,     0.59464,     0.77223,     0.75525,     0.78346,     0.20647,     0.65879,     0.53316,     0.69266,     0.60043])

In [35]:
# metrics.box.map    # map50-95
# metrics.box.map50  # map50
# metrics.box.map75  # map75
# metrics.box.maps 
validation_results.box.all_ap[:,-1]

array([   0.027735,    0.012984,   0.0093837,   0.0024506,   0.0018597,   0.0058105,    0.035251,    0.024297,     0.01992,    0.037053,     0.00579,    0.033996,    0.047395,   0.0098933,    0.055887,   0.0019691,    0.025987,    0.096861,   0.0062307,    0.016254])

In [7]:
!ls ../datasets/VOC/labels/test_orig

000001.txt  001641.txt	003328.txt  004959.txt	006656.txt  008324.txt
000002.txt  001644.txt	003329.txt  004964.txt	006659.txt  008325.txt
000003.txt  001645.txt	003332.txt  004965.txt	006662.txt  008328.txt
000004.txt  001646.txt	003333.txt  004969.txt	006663.txt  008330.txt
000006.txt  001648.txt	003334.txt  004970.txt	006665.txt  008331.txt
000008.txt  001652.txt	003340.txt  004971.txt	006669.txt  008333.txt
000010.txt  001655.txt	003341.txt  004975.txt	006672.txt  008334.txt
000011.txt  001656.txt	003342.txt  004978.txt	006675.txt  008337.txt
000013.txt  001657.txt	003345.txt  004979.txt	006676.txt  008339.txt
000014.txt  001658.txt	003346.txt  004980.txt	006680.txt  008340.txt
000015.txt  001659.txt	003347.txt  004981.txt	006683.txt  008343.txt
000018.txt  001660.txt	003348.txt  004988.txt	006685.txt  008344.txt
000022.txt  001663.txt	003352.txt  004989.txt	006686.txt  008347.txt
000025.txt  001664.txt	003353.txt  004993.txt	006688.txt  008348.txt
000027.txt  001665.txt	003357.txt 

In [37]:
class_names= [
        'aeroplane',
        'bicycle',
        'bird',
        'boat',
        'bottle',
        'bus',
        'car',
        'cat',
        'chair',
        'cow',
        'diningtable',
        'dog',
        'horse',
        'motorbike',
        'person',
        'pottedplant',
        'sheep',
        'sofa',
        'train',
        'tvmonitor'
    ]

directory = '../datasets/VOC/labels/test_orig'
count_class = {x:0 for x in range(20)}
files = os.listdir(directory)

for filename in files:
    with open (f'{directory}/{filename}', 'r') as f:
        data = f.readlines()
        classes_on_image = [x.split(' ')[0] for x in data]
        for cls_on_img in classes_on_image:
            count_class[int(cls_on_img)]+= 1



In [32]:
sum(count_class.values())

6307

In [38]:
dict(zip(class_names, count_class.values()))

{'aeroplane': 285,
 'bicycle': 337,
 'bird': 459,
 'boat': 263,
 'bottle': 469,
 'bus': 213,
 'car': 1201,
 'cat': 358,
 'chair': 756,
 'cow': 244,
 'diningtable': 206,
 'dog': 489,
 'horse': 348,
 'motorbike': 325,
 'person': 4528,
 'pottedplant': 480,
 'sheep': 242,
 'sofa': 239,
 'train': 282,
 'tvmonitor': 308}